# Solution to Problem 1

In [1]:
import pandas as pd
import numpy as np

In [2]:
matchdf = pd.read_csv('matches.csv')
delivdf = pd.read_csv('deliveries.csv')

In [3]:
matchdf.columns

Index(['id', 'season', 'city', 'date', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'result', 'dl_applied', 'winner', 'win_by_runs',
       'win_by_wickets', 'player_of_match', 'venue', 'umpire1', 'umpire2',
       'umpire3'],
      dtype='object')

In [4]:
delivdf.columns

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'non_striker', 'bowler', 'is_super_over', 'wide_runs',
       'bye_runs', 'legbye_runs', 'noball_runs', 'penalty_runs',
       'batsman_runs', 'extra_runs', 'total_runs', 'player_dismissed',
       'dismissal_kind', 'fielder'],
      dtype='object')

### 1) Which team has won/lost maximum number of matches while chasing?

In [5]:
t11 = matchdf[matchdf['winner']==matchdf['team2']]['team2']
t11.groupby(t11).count().idxmax()

'Kolkata Knight Riders'

In [6]:
t12 = matchdf[matchdf['winner']!=matchdf['team2']]['team2']
t12.groupby(t12).count().idxmax()

'Delhi Daredevils'

### 2) Which toss decision is best for each ground?

The value will be 'bat', 'field' or 'equal' depending on the number of matches that have been won by the batting or fielding team at that venue

In [7]:
t2 = matchdf.loc[:, ['winner','team1','team2','venue']]
# Add 'bat' and 'field' columns to the dataframe
t2['bat'] = (t2['team1'] == t2['winner'])
t2['field'] = (t2['team2'] == t2['winner'])
t2 = t2[['venue','bat','field']]
tt1 = pd.DataFrame(t2.groupby('venue')['bat'].apply(lambda x: x[x == True].count()))
tt2 = pd.DataFrame(t2.groupby('venue')['field'].apply(lambda x: x[x == True].count()))
t2sol = pd.merge(tt1, tt2, on='venue')
t2sol['decision'] = t2sol.apply (lambda row: ('bat' if row['bat']>row['field'] else 'equal') if row['bat']>=row['field'] else 'field', axis=1)
t2sol['decision']

venue
ACA-VDCA Stadium                                        field
Barabati Stadium                                          bat
Brabourne Stadium                                         bat
Buffalo Park                                              bat
De Beers Diamond Oval                                   field
Dr DY Patil Sports Academy                              field
Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium       bat
Dubai International Cricket Stadium                     field
Eden Gardens                                            field
Feroz Shah Kotla                                        field
Feroz Shah Kotla Ground                                 field
Green Park                                              field
Himachal Pradesh Cricket Association Stadium              bat
Holkar Cricket Stadium                                  field
IS Bindra Stadium                                       field
JSCA International Stadium Complex                      field
Ki

### 3) Which player has won maximum number of MoM awards while chasing?

In [8]:
t3 = matchdf[matchdf['winner']==matchdf['team2']]['player_of_match']
# Since the player of the match is almost always from the winning team
t3.groupby(t3).count().idxmax()

'YK Pathan'

### 4) Which batsman has played maximum number of dot balls?

In [9]:
t4 = delivdf.loc[:,['batsman','total_runs']]
t4[t4['total_runs']==0].groupby('batsman').count().idxmax()[0]

'V Kohli'

### 5) Which batsmen came to crease but never got a chance to face a ball?

In [10]:
t5 = delivdf.loc[:,['match_id','non_striker','batsman']].drop_duplicates()
for id, group in t5.groupby('match_id'):
    st1 = set(group['non_striker'])
    st2 = set(group['batsman'])
    st = st1-st2
    if len(st):
        print(id, '->', list(st1-st2))

2 -> ['MJ McClenaghan']
3 -> ['DR Smith']
4 -> ['R Bhatia']
10 -> ['A Nehra']
11 -> ['Sandeep Sharma']
21 -> ['CH Morris']
28 -> ['MJ McClenaghan', 'MG Johnson']
29 -> ['MK Pandey']
31 -> ['PJ Cummins']
37 -> ['S Aravind']
43 -> ['JD Unadkat', 'A Nehra']
45 -> ['S Aravind']
51 -> ['S Nadeem']
54 -> ['Sandeep Sharma']
64 -> ['PP Ojha']
71 -> ['AB Dinda']
74 -> ['JDP Oram']
75 -> ['I Sharma']
79 -> ['Joginder Sharma']
80 -> ['DP Vijaykumar', 'SB Bangar']
82 -> ['A Nehra']
90 -> ['A Kumble']
91 -> ['S Sreesanth']
95 -> ['GD McGrath']
109 -> ['A Nel']
111 -> ['Z Khan']
115 -> ['MM Patel']
118 -> ['PR Shah']
119 -> ['DW Steyn']
120 -> ['PP Chawla', 'T Kohli']
125 -> ['DL Vettori', 'L Balaji']
129 -> ['R Vinay Kumar']
130 -> ['K Goel']
131 -> ['MS Gony']
135 -> ['Z Khan']
138 -> ['AB Agarkar']
139 -> ['RR Powar']
140 -> ['Y Venugopal Rao']
141 -> ['S Tyagi', 'A Nehra']
148 -> ['A Kumble']
150 -> ['SL Malinga']
155 -> ['RP Singh']
157 -> ['PP Chawla']
158 -> ['PJ Sangwan']
159 -> ['S Tyagi']


### 6) Which bowler has hit for maximum number of 1s/2s/3s/4s/6s?

In [11]:
t6 = delivdf.loc[:,['bowler','total_runs']]
for i in [1,2,3,4,6]:
    print (i,'->' ,t6[t6['total_runs']==i].groupby('bowler').count().idxmax()[0])

1 -> Harbhajan Singh
2 -> DJ Bravo
3 -> P Kumar
4 -> UT Yadav
6 -> PP Chawla


### 7) Which batsman got stumped out maximum number of times?

In [12]:
t7 = delivdf.loc[:,['player_dismissed','dismissal_kind']]
t7 = t7[t7['dismissal_kind']=='stumped']['player_dismissed']
t7.groupby(t7).count().idxmax()

'SK Raina'

### 8) Which non-striker has been part of maximum number of runouts?

In [13]:
t8 = delivdf.loc[:,['non_striker','player_dismissed','dismissal_kind']]
t8 = t8[(t8['dismissal_kind']=='run out') & (t8['player_dismissed']==t8['non_striker'])]['player_dismissed']
t8.groupby(t8).count().idxmax()

'AB de Villiers'

### 9) Which bowler has bowled maximum number of dot balls?

In [14]:
t9 = delivdf.loc[:,['bowler','total_runs']]
t9[t9['total_runs']==0].groupby('bowler').count().idxmax()[0]

'Harbhajan Singh'

### 10) Which team holds the record for the maximum win by runs and against whom?

In [15]:
t10 = matchdf[['team1','team2','winner','win_by_runs']]
r10 = t10.iloc[t10['win_by_runs'].idxmax()]
print('{} won against {} by {} runs'.format(r10['team1'],r10['team2'],r10['win_by_runs']))

Mumbai Indians won against Delhi Daredevils by 146 runs


### 11) Which team holds the record for the maximum win by wickets and against whom?

In [16]:
t10 = matchdf.loc[:,['team1','team2','winner','win_by_wickets']]
t10['loser'] = t10.apply (lambda row: row['team1'] if row['team2']==row['winner'] else row['team1'], axis=1)
t10 = t10[t10['win_by_wickets']==10].loc[:,['winner','loser']]
t10

,winner,loser
2,Kolkata Knight Riders,Gujarat Lions
34,Kings XI Punjab,Delhi Daredevils
71,Deccan Chargers,Mumbai Indians
119,Delhi Daredevils,Kings XI Punjab
183,Royal Challengers Bangalore,Rajasthan Royals
298,Rajasthan Royals,Mumbai Indians
376,Mumbai Indians,Rajasthan Royals
390,Chennai Super Kings,Kings XI Punjab
542,Royal Challengers Bangalore,Delhi Daredevils
590,Sunrisers Hyderabad,Gujarat Lions


### 12) Which fielder took most catches in IPL?

In [17]:
t11 = delivdf.loc[:,['fielder','dismissal_kind']]
t11 = t11[t11['dismissal_kind']=='caught']['fielder']
t7.groupby(t7).count().idxmax()

'SK Raina'

### 13) Which umpire officiated the most matches?

In [18]:
t12 = matchdf.loc[:,['umpire1','umpire2','umpire3']]
pd.concat([t12['umpire1'],t12['umpire2'],t12['umpire3']],ignore_index=True).value_counts().idxmax()

'S Ravi'